In [ ]:
# Instalação para vEnv
%pip install openai

In [7]:
import os
import pickle
import time
from openai import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter

os.environ['OPENAI_API_KEY'] = "YOUR_KEY_PLEASE"

# 2. Extrair Visões e Propostas dos textos dos planos de governo


In [8]:
#Carregar documentos preprocessados
with open('STD_documents.pkl', 'rb') as f:
    all_docs = pickle.load(f)

In [15]:
num_chars=0
num_words = 0
for doc in all_docs:
    st = str(doc.page_content)
    num_chars = num_chars + len(st)
    words = st.split()
    num_words = num_words + len(words)

print("Number of words:", num_words)
print("Number of chars:", num_chars)

Number of words: 107988
Number of chars: 720957


In [3]:
# Chave para acesso ao API da OepnAI
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=OPENAI_API_KEY)

In [8]:
# Decorator para medição de tempos
def timeit(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"  - tempo de operação {end_time - start_time:.1f}s")
        return result
    return wrapper

# Chamada ao GPT 4 Turbo
@timeit
def extract_information(text, doc_id, model="gpt-4o-mini"):
   completion = client.chat.completions.create(
        model=model, # Gpt 4 é um excelente custo beneficio para este uso
        temperature=0, # Não queremos criatividade, apenas análise semântica
        messages=[
        {
            "role": "system",
            "content": system_prompt # Parâmetro para prompt
        },
        {
            "role": "user",
            "content": user_prompt.format(
              entity_types=node_types,
              specification="## Documento:" + doc_id + "\n## Conteúdo:" + text
            ) # Parâmetros para composição do prompt
        }
        ]
    )

   return completion.choices[0].message.content, completion.usage.total_tokens


# Definição da função do assitente
system_prompt = """
Você é um agente especializado em analizar documentos que descrevem planos de governo e contêm, entre outras informações, os valores e as propostas dos candidatos ao cargo de prefeito de Curitiba.
Você recebe trechos dos documentos identificado com um DOC_ID. Sua tarefa é identifocar em cada document as propostas e os valores citados (node), bem como se existe relações entre eles (link).
Você deve gerar o OUTPUT em formato JSON. Este JSON deve conter objetos com as seguintes chaves:
"item_type", que é a chave que identifica se o objeto se refere a um nó (node) ou a uma ligação (link).
Se o objeto for um nó, deve conter a chave "node_name", com o nome do nó, "node_type" com o tipo de nó e "node_description", com a descrição do nó.
Se o objeto for uma ligação, deve trazer os nomes dos nós que são relacionados, usando as chaves "head_node" e "tail_node".
Tente sempre extrair o maior número de valores e propostas contidos documento fornecido.
A linguagem a ser sempre utilizada é o Português do Brasil."""

# Definição da estrutura do prompt
user_prompt = """
Baseado no exemplo fornecido, extraia os itens, ou seja, os valores, propostas e ligações contidos nos documentos DOC_ID fornecidos.
Como definição de valor usar: "Apreço e estimação: Valor é o apreço e estimação que se tem por algo ou alguém. Normas sociais: Valores são normas ou padrões sociais que orientam a ação humana e são geralmente aceitos por um indivíduo, classe ou sociedade. Pode descrever visões, anseios e desejos de quem os tem."
Como definição de proposta usar: "Aquilo que se está prometendo ou sugerindo que seja realizado."

--> Início do Exemplo:

## Documento: NICHO04
## Conteúdo: É preciso mudar o atual cenário de abandono de nosso município, para tanto tem
que se colocar em prática o plano diretor, e dos instrumentos nele previstos para o uso
ordenado do solo urbano, com obras de infraestrutura que proporcionem um melhor
escoamento das águas pluviais, evitando as enchentes em épocas de chuvas. Elaborar
projetos para a reestruturação do sistema viário com readequação das vias que dão
acesso a PR-407, acabando com as longas filas de veículos nos feriadões, e em época
de temporada.
Modernizar o sistema de transporte coletivo no município, fazendo com que as
empresas façam atendimento também ao interior dos principais balneários e colônias, e
não somente pela PR-412, além de fazer a integração com os municípios de Matinhos e
Paranaguá, em razão do grande número de usuários, estudantes, trabalhadores e
turistas que se deslocam diariamente para estes municípios, com uma tarifa social justa.
Construção de um terminal rodoviário municipal moderno, capaz de atender a população,
visitantes e aos turistas.
Envidar esforços com o objetivo de municipalizar a PR 412, transformando-a em
uma avenida revitalizada, moderna, com arborização, estacionamentos, infraestrutura de
escoamento de águas pluviais, galerias, manilhamentos, boca de lobo, evitando as
enchentes em épocas de grandes chuvas.
Revitalização da Av. Beira Mar, com calçadas padronizadas, ciclovias, iluminação
moderna, arborização, cuidado com a restinga, restauração dos banheiros que estão
abandonados, e construções de novos, nos balneários, manutenção e construção dos
decks de acesso a praia em todos os balneários.
Obras planejadas com o objetivo de escoamento das águas pluviais das vias
públicas; limpezas periódicas e desassoreamento dos canais desenvolver esforços junto
ao Estado e a União com o escopo de concluir as obras de saneamento básico no
município, posto que Pontal do Paraná é um município voltado para o turismo de veraneio
e praias.
Colocar placas de identificação nas vias públicas, e de identificação nos balneários
na Avenida Beira Mar.
Construir portais modernos nas divisas com Matinhos, e Paranaguá, acabando
com a sensação de abandono em que o município se encontra.
Fazer obras de pavimentação asfáltica e calçamentos padronizados nas ruas e
avenidas, com colocação de semáforos, proporcionando mais segurança aos usuários.


## Output
[
  {{
    "item_type": "node",
    "node_name": "Mudar o atual cenário de abandono do município.",
    "node_type": "valor",
    "node_description": "É preciso mudar o atual cenário de abandono de nosso município, para tanto tem que se colocar em prática o plano diretor, e dos instrumentos nele previstos para o uso ordenado do solo urbano, com obras de infraestrutura que proporcionem um melhor escoamento das águas pluviais, evitando as enchentes em épocas de chuvas."
  }},
  {{
    "item_type": "node",
    "node_name": "Reestruturação do sistema viário de acesso à PR-407.",
    "node_type": "proposta",
    "node_description": "Elaborar projetos para a reestruturação do sistema viário com readequação das vias que dão acesso a PR-407, acabando com as longas filas de veículos nos feriadões, e em época de temporada."
  }},
  {{
    "item_type": "link",
    "head_node": "Reestruturação do sistema viário de acesso à PR-407.",
    "tail_node": "Mudar o atual cenário de abandono do município."
  }},
  {{
    "item_type": "node",
    "node_name": "Modernização do o sistema de transporte coletivo.",
    "node_type": "proposta",
    "node_description": "Modernizar o sistema de transporte coletivo no município, fazendo com que as empresas façam atendimento também ao interior dos principais balneários e colônias, e não somente pela PR-412."
  }},
  {{
    "item_type": "node",
    "node_name": "Fazer a integração com os municípios de Matinhos e Paranaguá.",
    "node_type": "proposta",
    "node_description": "Fazer a integração com os municípios de Matinhos e Paranaguá, em razão do grande número de usuários, estudantes, trabalhadores e turistas que se deslocam diariamente para estes municípios, com uma tarifa social justa."
  }},
  {{
    "item_type": "link",
    "head_node": "Fazer a integração com os municípios de Matinhos e Paranaguá.",
    "tail_node": "Modernização do o sistema de transporte coletivo."
  }},
  {{
    "item_type": "node",
    "node_name": "Construção de um novo terminal rodoviário.",
    "node_type": "proposta",
    "node_description": "Construção de um terminal rodoviário municipal moderno, capaz de atender a população, visitantes e aos turistas."
  }},
  {{
    "item_type": "link",
    "head_node": "Construção de um novo terminal rodoviário.",
    "tail_node": "Modernização do o sistema de transporte coletivo."
  }},
  {{
    "item_type": "node",
    "node_name": "Municipalização da PR 412.",
    "node_type": "proposta",
    "node_description": "Envidar esforços com o objetivo de municipalizar a PR 412, transformando-a em uma avenida revitalizada, moderna, com arborização, estacionamentos, infraestrutura de escoamento de águas pluviais, galerias, manilhamentos, boca de lobo, evitando as enchentes em épocas de grandes chuvas."
  }},
  {{
    "item_type": "node",
    "node_name": "Revitalização da Av. Beira Mar.",
    "node_type": "proposta",
    "node_description": "Revitalização da Av. Beira Mar, com calçadas padronizadas, ciclovias, iluminação moderna, arborização, cuidado com a restinga, restauração dos banheiros que estão abandonados, e construções de novos, nos balneários, manutenção e construção dos decks de acesso a praia em todos os balneários."
  }},
  {{
    "item_type": "node",
    "node_name": "Conclusão de obras de saneamento básico.",
    "node_type": "proposta",
    "node_description": "Obras planejadas com o objetivo de escoamento das águas pluviais das vias públicas; limpezas periódicas e desassoreamento dos canais desenvolver esforços junto ao Estado e a União com o escopo de concluir as obras de saneamento básico no município, posto que Pontal do Paraná é um município voltado para o turismo de veraneio e praias."
  }},
  {{
    "item_type": "node",
    "node_name": "Identificação de vias públicas.",
    "node_type": "proposta",
    "node_description": "Colocar placas de identificação nas vias públicas, e de identificação nos balneários na Avenida Beira Mar."
  }},
  {{
    "item_type": "node",
    "node_name": "Identificação das divisas.",
    "node_type": "proposta",
    "node_description": "Construir portais modernos nas divisas com Matinhos, e Paranaguá, acabando com a sensação de abandono em que o município se encontra."
  }},
  {{
    "item_type": "node",
    "node_name": "Pavimentação e sinalização das ruas.",
    "node_type": "proposta",
    "node_description": "Fazer obras de pavimentação asfáltica e calçamentos padronizados nas ruas e avenidas, com colocação de semáforos, proporcionando mais segurança aos usuários."
  }}
]

--> Fim do exemplo

Para o documento a seguir, extrair as propstas, valores e quais propostas e valores são relacionadas.
{specification}

## Output
"""

#find first '[' and last ']' in string
# Esta função apenas verifica se o GPT retornou uma lista ou um text, transformando texto para lista
import json

def find_brackets(string):
    first = 0
    for i in range(len(string)):
        if string[i] == '[' and first==0:
            first = i
        if string[i] == ']':
            last = i
    return first, last

# transform text to json
def text_to_json(text):
    json_data = json.loads(text)
    return json_data

def ajustar_output(output):
    if not isinstance(output, list):
        f, l = find_brackets(output)
        return text_to_json(output[f:l+1])
    return output

In [ ]:
# EXTRAIR

text_splitter = RecursiveCharacterTextSplitter(separators=[".\n"],chunk_size = 2000, chunk_overlap=0)

final_json = []
for doc in all_docs:
    chunked_docs = text_splitter.split_documents([doc])
    n=1
    for chunked_doc in chunked_docs:
        print("doc_id", doc.id, "- Chunk #", n, "de", len(chunked_docs))
        extracted_relations, tokens_used = extract_information(chunked_doc.page_content, doc_id)
        extracted_relations = ajustar_output(extracted_relations)
        print(tokens_used, "tokens usados para extrair", len(extracted_relations), "relações")
        final_json.append({"doc_id": doc.id, "elements": extracted_relations})
        n = n + 1


doc_id ANDRE00 - Chunk # 1 de 2
  - tempo de operação 15.4s
3387 tokens usados para extrair 13 relações
doc_id ANDRE00 - Chunk # 2 de 2
  - tempo de operação 6.0s
2542 tokens usados para extrair 5 relações
doc_id ANDRE01 - Chunk # 1 de 14
  - tempo de operação 9.1s
2956 tokens usados para extrair 7 relações
doc_id ANDRE01 - Chunk # 2 de 14
  - tempo de operação 14.0s
3153 tokens usados para extrair 9 relações
doc_id ANDRE01 - Chunk # 3 de 14
  - tempo de operação 13.2s
3114 tokens usados para extrair 12 relações
doc_id ANDRE01 - Chunk # 4 de 14
  - tempo de operação 15.6s
3281 tokens usados para extrair 12 relações
doc_id ANDRE01 - Chunk # 5 de 14
  - tempo de operação 14.6s
3266 tokens usados para extrair 12 relações
doc_id ANDRE01 - Chunk # 6 de 14
  - tempo de operação 13.6s
3125 tokens usados para extrair 9 relações
doc_id ANDRE01 - Chunk # 7 de 14
  - tempo de operação 15.6s
3106 tokens usados para extrair 9 relações
doc_id ANDRE01 - Chunk # 8 de 14
  - tempo de operação 16.3s
317

In [ ]:
#limpar caracteres inválidos no final_json (aspas simples e duplas causam falhas no carregamento do grafo)
for i in range(len(final_json)):
    for j in range(len(final_json[i]['elements'])):
        if final_json[i]['elements'][j]['item_type'] == 'node':
            final_json[i]['elements'][j]['node_name'] = final_json[i]['elements'][j]['node_name'].replace('"', '')
            final_json[i]['elements'][j]['node_name'] = final_json[i]['elements'][j]['node_name'].replace("'", "")
            final_json[i]['elements'][j]['node_description'] = final_json[i]['elements'][j]['node_description'].replace('"', '')
            final_json[i]['elements'][j]['node_description'] = final_json[i]['elements'][j]['node_description'].replace("'", "")
        else:
            final_json[i]['elements'][j]['head_node'] = final_json[i]['elements'][j]['head_node'].replace('"', '')
            final_json[i]['elements'][j]['head_node'] = final_json[i]['elements'][j]['head_node'].replace("'", "")
            final_json[i]['elements'][j]['tail_node'] = final_json[i]['elements'][j]['tail_node'].replace('"', '')
            final_json[i]['elements'][j]['tail_node'] = final_json[i]['elements'][j]['tail_node'].replace("'", "")

#limpar caracteres invalidos no all_docs

for i in range(len(all_docs)):
    all_docs[i].page_content = all_docs[i].page_content.replace('"', '')
    all_docs[i].page_content = all_docs[i].page_content.replace("'", "")

In [ ]:
# Salvar informação extraída

with open('entidades_full.pkl', 'wb') as f:
    pickle.dump(final_json, f)